In [2]:
import pandas as pd
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
animeDf = pd.read_csv('anime.csv')

In [4]:
def getAnimeName(id):
	name = animeDf[animeDf.MAL_ID == id]['English name'].values[0]
	if name is np.nan or name == "Unknown":
		name = animeDf[animeDf.MAL_ID == id]['Name'].values[0]
	return name
animeDf['eng_name'] = animeDf.MAL_ID.apply(lambda x :getAnimeName(x) )

In [5]:

animeDf = animeDf[['Score','Ranked','Genres','eng_name','Type','Episodes','Source','Studios']]
animeDf = animeDf[animeDf.Score != 'Unknown']
animeDf = animeDf[animeDf.Ranked!= 'Unknown']
animeDf = animeDf[animeDf.Episodes != 'Unknown']

In [6]:
animeDf.Score = animeDf.Score.apply(lambda x: float(x))
animeDf.Ranked = animeDf.Ranked.apply(lambda x: float(x))
animeDf.Episodes = animeDf.Episodes.apply(lambda x: int(x))

In [7]:
animeDf

,Score,Ranked,Genres,eng_name,Type,Episodes,Source,Studios
0,8.78,28.0,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,TV,26,Original,Sunrise
1,8.39,159.0,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,Movie,1,Original,Bones
2,8.24,266.0,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,TV,26,Manga,Madhouse
3,7.27,2481.0,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,TV,26,Original,Sunrise
4,6.98,3710.0,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,TV,52,Manga,Toei Animation
...,...,...,...,...,...,...,...,...
17504,6.59,5461.0,Comedy,Kimetsu Gakuen: Valentine-hen,ONA,4,Original,Unknown
17505,7.52,1517.0,"Music, Romance",Heikousen,Music,1,Original,"10Gauge, Studio DURIAN"
17512,6.83,4382.0,"Slice of Life, Comedy",Cooking with Valkyries Spring Festival Special,Special,1,Game,Unknown
17513,4.81,10760.0,"Mystery, Psychological, Supernatural, Thriller...",The Promised Neverland Season 2 Episode 5.5,Special,1,Manga,CloverWorks


In [8]:
l = animeDf.Genres.to_numpy()
l= [x.split(',') for x in l]
uniqueGenres = []
for x in l:
	for y in x:
		y.strip()
		if y not in uniqueGenres:
			uniqueGenres.append(y)
uniqueGenres = np.array(uniqueGenres)
uniqueGenres = [x.strip() for x in uniqueGenres]
uniqueGenres = np.unique(uniqueGenres)
genreCol = animeDf.Genres
index = animeDf.index
t = []
for x in genreCol:
	ohe = np.zeros(len(uniqueGenres))
	genres = x.split(',')
	genres = [y.strip() for y in genres]
	for y in genres:
		index = np.where(uniqueGenres == y)
		ohe[index[0][0]] = 1
	t.append(ohe)
#t = pd.Series(t, name='Genres', index=animeDf.index)
genresDf = pd.DataFrame(t, columns=uniqueGenres, index=animeDf.index)

In [10]:
genresDf.drop('Unknown', inplace=True, axis=1)
genresDf.head()

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire
0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [37]:
from sklearn.preprocessing import LabelEncoder
ndf = animeDf.copy()

le = LabelEncoder()
le.fit(ndf.Type)
ndf.Type = le.transform(ndf.Type)

le.fit(ndf.Source)
ndf.Source = le.transform(ndf.Source)

le.fit(ndf.Studios)
ndf.Studios = le.transform(ndf.Studios)

target = ndf.pop('Score')
ndf.drop(columns='Genres',inplace=True)
ndf = ndf.join(genresDf)
ndf.drop(columns=['Ranked','eng_name'], inplace=True)


In [38]:
ndf.head()

,Type,Episodes,Source,Studios,Action,Adventure,Cars,Comedy,Dementia,Demons,...,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire
0,5,26,9,718,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0,1,9,120,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,5,26,6,384,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,26,9,718,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,52,6,792,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [39]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(ndf, target, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [60]:
from sklearn.ensemble import RandomForestRegressor


# Define the models
model_1 = RandomForestRegressor(n_estimators=300, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
# model_3 = RandomForestRegressor(n_estimators=100, criterion='absolute_error', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)
models = [model_1, model_2, model_4, model_5]

In [61]:
from sklearn.metrics import mean_absolute_error
model_1.fit(X_train, y_train)
preds = model_1.predict(X_valid)
print(mean_absolute_error(y_valid, preds))
model_2.fit(X_train, y_train)
preds = model_2.predict(X_valid)
print(mean_absolute_error(y_valid, preds))
model_4.fit(X_train, y_train)
preds = model_4.predict(X_valid)
print(mean_absolute_error(y_valid, preds))
model_5.fit(X_train, y_train)
preds = model_5.predict(X_valid)
print(mean_absolute_error(y_valid, preds))

0.46231178842699927
0.4644680818757707
0.47986277159885815
0.5285702538835951
